In [2]:
from glob import glob
import os
import shutil
import nibabel as nib
from scipy import ndimage
from seg_brain_mri import main as get_mask
import torchmetrics
import torch
import numpy as np

In [14]:
#from torchmetrics.segmentation import HausdorffDistance
#from torchmetrics.functional.segmentation.hausdorff_distance import hausdorff_distance
from monai.metrics import compute_hausdorff_distance

data_dir = os.path.join('dataset3/test/')
mris = sorted(glob(os.path.join(data_dir, "*.nii.gz")))
outdatasets = ['IRIS', 
               'Feminad', 
                'GIN', 
              ]
models = [
         'old_final/seg/brain_resample_iris.pth',
         'old_final/seg/brain_resample_feminad.pth',
         'old_final/seg/brain_resample_irisfeminad.pth',
         ]

#dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=2)
dicemetric = torchmetrics.JaccardIndex(task='multiclass', average=None, mdmc_average='global', num_classes=2)


for i, model in enumerate(models): 
    for outdataset in outdatasets:    
        
        mris_raw = sorted(glob(os.path.join('dataset3', outdataset, 'MRI', "*_id.nii.gz")))
        truths = sorted(glob(os.path.join('dataset3', outdataset, 'Mask', "*_id.nii.gz")))  
        bigdice = []
        num_int = 0
        for j, mri in enumerate(mris_raw):
            if outdataset == 'GIN' or (outdataset == 'Feminad' and j > 20+6) or (outdataset == 'IRIS' and j > 248+82) or (outdataset == 'Painfact' and j > 169+56):
                pred = torch.from_numpy(get_mask(model, mri, "", False, False, False, False).numpy()).type('torch.LongTensor').unsqueeze(0).unsqueeze(0)
                truth = torch.from_numpy(nib.load(truths[j]).get_fdata()).type('torch.LongTensor').unsqueeze(0).unsqueeze(0)
                #dice = dicemetric(pred, truth)
                dice = compute_hausdorff_distance(pred, truth).item()
                #print(dice)
                #dice = dice[1].item()
                #dice = dice.item()
                bigdice.append(dice)
                num_int += 1
        mean = np.mean(bigdice)
        std = np.std(bigdice) 
        print(outdataset + ' - ' + model.split('_')[-1].split('.')[0] + " : {:.4f} - {:.4f}".format(mean, std))

IRIS - iris : 18.1705 - 11.6843
Feminad - iris : 15.8917 - 2.9075
GIN - iris : 15.6264 - 11.7600
IRIS - feminad : 28.8673 - 13.5095
Feminad - feminad : 18.5578 - 3.8439
GIN - feminad : 20.4041 - 7.9469
IRIS - irisfeminad : 20.2339 - 13.0273
Feminad - irisfeminad : 19.2270 - 8.2456
GIN - irisfeminad : 15.1778 - 11.8386


dice:

IRIS - iris : 0.9651 - 0.0172
Feminad - iris : 0.9586 - 0.0206
GIN - iris : 0.9452 - 0.0206
IRIS - feminad : 0.8938 - 0.0731
Feminad - feminad : 0.9511 - 0.0121
GIN - feminad : 0.9267 - 0.0316
IRIS - irisfeminad : 0.9660 - 0.0166
Feminad - irisfeminad : 0.9589 - 0.0164
GIN - irisfeminad : 0.9581 - 0.0131

iou:
IRIS - iris : 0.9331 - 0.0308
Feminad - iris : 0.9212 - 0.0368
GIN - iris : 0.8967 - 0.0361
IRIS - feminad : 0.8155 - 0.1123
Feminad - feminad : 0.9069 - 0.0216
GIN - feminad : 0.8650 - 0.0537
IRIS - irisfeminad : 0.9346 - 0.0300
Feminad - irisfeminad : 0.9215 - 0.0297
GIN - irisfeminad : 0.9199 - 0.0234

hausdorff:
IRIS - iris : 18.1705 - 11.6843
Feminad - iris : 15.8917 - 2.9075
GIN - iris : 15.6264 - 11.7600

IRIS - feminad : 28.8673 - 13.5095
Feminad - feminad : 18.5578 - 3.8439
GIN - feminad : 20.4041 - 7.9469

IRIS - irisfeminad : 20.2339 - 13.0273
Feminad - irisfeminad : 19.2270 - 8.2456
GIN - irisfeminad : 15.1778 - 11.8386




In [15]:
def iou(dice):
    iou = dice / (2 - dice)
    return iou

print("{:.4f}".format(iou(0.0172)))

0.0087


In [ ]:
data_dir = os.path.join('dataset3/test/')
mris = sorted(glob(os.path.join(data_dir, "*.nii.gz")))
outdatasets = ['GIN', 
               'Feminad', 
               'IRIS', 
               #'Painfact',
              ]
models = [#'final/seg/brain_resample_painfactirisfeminad.pth',
         'final/seg/brain_resample_iris.pth',
         'final/seg/brain_resample_feminad.pth',
         'final/seg/brain_resample_irisfeminad.pth',
         #'final/seg/brain_resample_painfact.pth',
         ]

dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=2)

for outdataset in outdatasets:
    for i, model in enumerate(models):     
        
        mris_raw = sorted(glob(os.path.join('dataset3', outdataset, 'MRI', "*_id.nii.gz")))
        truths = sorted(glob(os.path.join('dataset3', outdataset, 'Mask', "*_id.nii.gz")))  
        bigdice = []
        num_int = 0
        for j, mri in enumerate(mris_raw):
            if outdataset == 'GIN' or (outdataset == 'Feminad' and j > 20+6) or (outdataset == 'IRIS' and j > 248+82) or (outdataset == 'Painfact' and j > 169+56):
                pred = torch.from_numpy(get_mask(model, mri, "", False, False, False, False).numpy().ravel()).type('torch.LongTensor')
                truth = torch.from_numpy(nib.load(truths[j]).get_fdata().ravel()).type('torch.LongTensor')
                dice = dicemetric(pred, truth)[1].item()
                bigdice.append(dice)
                num_int += 1
        mean = np.mean(bigdice)
        std = np.std(bigdice) 
        print("{:.4f} - {:.4f} : {:.4f} - {:.4f}".format(outdataset, model.split('_')[-1], mean, std))
        print(outdataset + ' - ' + model.split('_')[-1] + ' : ' + str(mean) + ' - ' + str(std))

In [ ]:
GIN - iris.pth :            0.9451652610743487 - 0.020613108146719464
GIN - feminad.pth :         0.9266778252742909 - 0.03159560840199229
GIN - irisfeminad.pth :     0.9581366954026399 - 0.01307884589202915
Feminad - iris.pth :        0.9586123724778494 - 0.020581658353281594
Feminad - feminad.pth :     0.9510500927766165 - 0.01205309508080402
Feminad - irisfeminad.pth : 0.9588670333226522 - 0.01643535535143678
IRIS - iris.pth :           0.9651128193478525 - 0.01715841335326685
IRIS - feminad.pth :        0.8938481174869302 - 0.07306338569929116
IRIS - irisfeminad.pth :    0.965963097266209  - 0.016616582535859833

In [ ]:
GIN - painfactirisfeminad.pth : 0.9612319381148727
GIN - iris.pth : 0.9451652610743487
GIN - irisfeminad.pth : 0.9581366954026399
GIN - feminad.pth : 0.9266778252742909
GIN - painfact.pth : 0.9396339588695102
Feminad - painfactirisfeminad.pth : 0.9605755011240641
Feminad - iris.pth : 0.9586123724778494
Feminad - irisfeminad.pth : 0.9588670333226522
Feminad - feminad.pth : 0.9510500927766165
Feminad - painfact.pth : 0.9536308745543162
IRIS - painfactirisfeminad.pth : 0.9637849455998267
IRIS - iris.pth : 0.9651128193478525
IRIS - irisfeminad.pth : 0.965963097266209
IRIS - feminad.pth : 0.8938481174869302
IRIS - painfact.pth : 0.9117101857691635
Painfact - painfactirisfeminad.pth : 0.9796699821949005
Painfact - iris.pth : 0.9644888246059418
Painfact - irisfeminad.pth : 0.9716401553153992
Painfact - feminad.pth : 0.9687978184223175
Painfact - painfact.pth : 0.9813210034370422